### Some background info:
**Cohort Analysis <span class="mark">同期群体分析</span>是一种非常重要的分析手段，它能把<span class="mark">客户的留存情况及活跃情况可视化</span>。常见的Cohort Analysis比较喜欢以注册时间来对客户进行一个划分，比如以用户第一次建立账号开始消费的时间来划分客户群，借此来分析这些客户在3个月、6个月甚至一年后的留存情况**

This notebook file is dedicated in conducting cohort analysis on commonly seen sales data.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
# Import the data and inspect on it:

df = pd.read_csv("online.csv")
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,10/25/11 8:26,2.10,14286,United Kingdom
1,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,11/20/11 11:56,1.45,16360,United Kingdom
2,560034,23299,FOOD COVER WITH BEADS SET 2,6,7/14/11 13:35,3.75,13933,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70864 entries, 0 to 70863
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    70864 non-null  int64  
 1   StockCode    70864 non-null  object 
 2   Description  70864 non-null  object 
 3   Quantity     70864 non-null  int64  
 4   InvoiceDate  70864 non-null  object 
 5   UnitPrice    70864 non-null  float64
 6   CustomerID   70864 non-null  int64  
 7   Country      70864 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 4.3+ MB


**Two things:**

**1)We have two separate columns for UnitPrice and Quantity, thus, we need to combine them to a total amount of the order. ie. InvoiceNo
2)InvoiceDate is in object type, we need it in datetime format, and also, get rid of the the hours and minutes.**



In [6]:
# Get the sum column for each order
df['InvoiceSum'] = df['UnitPrice']*df['Quantity']

# Get the datetime format for InvoiceDate
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

**Sidenote:**

这里我们需要按照月份数做Cohort Analysis。那么我们除了无视时间，也可以无视days，我们可以create a function that takes in a date and returns only the year, month and day=1 of it. 设置天数是1的原因是因为我们只需要年份和月份，所以所有的days都可以默认为当月的1号，而且date和datime function必须要year，month和day三个attributes都有。

In [7]:
# Create a function that will take a datetime object as input and output only the year, month and day of it:

def get_date(x):
    date = dt.date(x.year, x.month, 1)
    return date

# Apply the function to the InvoiceDate column by using the apply() function.
# sidenote: apply() passes a function along an axis of the DataFrame.

df['InvoiceDate'] = df['InvoiceDate'].apply(get_date)

x = df['InvoiceDate'].apply(get_date)

x

0        2011-10-01
1        2011-11-01
2        2011-07-01
3        2011-11-01
4        2011-05-01
            ...    
70859    2011-11-01
70860    2010-12-01
70861    2011-07-01
70862    2011-05-01
70863    2011-08-01
Name: InvoiceDate, Length: 70864, dtype: object

**Sidenote on groupby + transform:**

**groupby+aggregation会返回数据的缩减版本，<span class="mark">而transform能返回完整数据的某一变换版本供重组</span>。这样的transformation，<span class="mark">输出的形状和输入一致</span>。一个常见的例子是通过减去分组平均值来居中数据。常见用法为先按照分组groupby之后选定需要操作的column接上.tansform('func')。transform会return和原data同样长度的Series并且在原长度的Series里按照分组的variable进行func中的操作。**

**此处我们可以利用transform来找到每一个CustomerID第一次下单的日期：**

In [8]:
# We first groupby CustomerID, then apply min func on the InvoiceDate to each group of IDs. transform() will return
# a pd.Series with the same length of the original dataframe b4 groupby.

df['CohortDate'] = df.groupby('CustomerID')['InvoiceDate'].transform('min')

**Progress Check:**

**我们现在有两列日期，一列是每一个订单的日期，而CohortDate则对应每一个ID第一次下单的日期。我们现在可以继续Cohort Analysis的下一步：<span class="mark">计算每一个订单的日期(InvoiceDate)和CohortDate之间差了多少个月</span>。这样做的目的是因为在留存分析中，我们需要探究顾客在第一次消费之后的一段时间，比如一年以内，再次进行消费的频率和概率以便更好的知道我们的客户的留存情况。**

In [11]:
# Create functions that return the year and the month for a datetime object:
def get_year(x):
    year = x.year
    return year

def get_month(x):
    month = x.month
    return month

# Apply funcs on InvoiceDate and CohortDate

invoice_year = df['InvoiceDate'].apply(get_year)
invoice_month = df['InvoiceDate'].apply(get_month)

cohort_year = df['CohortDate'].apply(get_year)
cohort_month = df['CohortDate'].apply(get_month)

# Now we can calculate the difference between each pair of invoice date and cohort date in years and months:

year_diff = invoice_year - cohort_year 
month_diff = invoice_month - cohort_month
total_diff = year_diff*12+month_diff

df['Diff'] = total_diff

df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CohortDate,InvoiceSum,Diff
0,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-01,2.10,14286,United Kingdom,2011-04-01,12.60,6
1,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-01,1.45,16360,United Kingdom,2011-09-01,1.45,2
2,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-01,3.75,13933,United Kingdom,2011-07-01,22.50,0


**Progress Check:**

**我们现在知道了每单和对应的Cohort Date的差距是months，那么现在就可以build up Cohort Table。Cohort Table的index应为每一个unique的月份，而columns则是从1开始依次增大开始排列，意思是从每一个月开始之后的几个月内客户的再次消费情况。**

**One tool to use is the 'nunique': nunique()与unique()十分类似，会return the number of unique counts。这里按照CohortDate和Diff grouping之后在agg里针对CustomerID输入nunique会针对每个CohortDate和Diff的pair返回number of unique CustomerID。**

**这里的逻辑是我们在每个日期和Diff的pair中查看有多少个unique的ID下过订单。在每一个CohortDate中当Diff是0的时候，就是多少个unique的ID在这个日期下过订单，当Diff等于1时，就是在同一个CohortDate下单的ID里有多少在第二个月再次下了订单。这便是Customer的retention，就是针对在同一个Cohort月份第一次下订单的ID里有多少是在第二个月及之后的每一个月里留存的客户。**

In [84]:
grouping = df.groupby(['CohortDate', 'Diff']).agg({'CustomerID' : 'nunique'}).reset_index()

In [85]:
grouping

,CohortDate,Diff,CustomerID
0,2010-12-01,0,716
1,2010-12-01,1,246
2,2010-12-01,2,221
3,2010-12-01,3,251
4,2010-12-01,4,245
...,...,...,...
86,2011-10-01,1,67
87,2011-10-01,2,30
88,2011-11-01,0,291
89,2011-11-01,1,32


In [86]:
grouping.pivot(index = 'CohortDate', columns = 'Diff', values = 'CustomerID')

Diff,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortDate,,,,,,,,,,,,,
2010-12-01,716.0,246.0,221.0,251.0,245.0,285.0,249.0,236.0,240.0,265.0,254.0,348.0,172.0
2011-01-01,332.0,69.0,82.0,81.0,110.0,90.0,82.0,86.0,104.0,102.0,124.0,45.0,NaN
2011-02-01,316.0,58.0,57.0,83.0,85.0,74.0,80.0,83.0,86.0,95.0,28.0,NaN,NaN
2011-03-01,388.0,63.0,100.0,76.0,83.0,67.0,98.0,85.0,107.0,38.0,NaN,NaN,NaN
2011-04-01,255.0,49.0,52.0,49.0,47.0,52.0,56.0,59.0,17.0,NaN,NaN,NaN,NaN
2011-05-01,249.0,40.0,43.0,36.0,52.0,58.0,61.0,22.0,NaN,NaN,NaN,NaN,NaN
2011-06-01,207.0,33.0,26.0,41.0,49.0,62.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07-01,173.0,28.0,31.0,38.0,44.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08-01,139.0,30.0,28.0,35.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
